# Data Setup and Processing

This notebook handles:
1. Decompressing GZ files to JSON
2. Converting JSON files to Parquet in chunks

In [6]:
!pip install tqdm
import duckdb
import os
import gzip
import glob
import shutil

# Create necessary directories
os.makedirs('/data/json', exist_ok=True)
os.makedirs('/data/parquet', exist_ok=True)


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


## 1. Decompress GZ Files

In [2]:
def decompress_gz_files():
    gz_files = glob.glob('/data/elasticsearch/*.json.gz')
    
    for gz_file in gz_files:
        base_name = os.path.basename(gz_file)
        json_name = base_name[:-3]  # Remove .gz extension
        json_path = os.path.join('/data/json', json_name)
        
        print(f"🔄 Decompressing {base_name}...")
        
        with gzip.open(gz_file, 'rb') as f_in:
            with open(json_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
                
        print(f"✅ Created {json_name}")

decompress_gz_files()

🔄 Decompressing aarecords__11.json.gz...
✅ Created aarecords__11.json
🔄 Decompressing aarecords__1.json.gz...
✅ Created aarecords__1.json
🔄 Decompressing aarecords__8.json.gz...
✅ Created aarecords__8.json
🔄 Decompressing aarecords__6.json.gz...
✅ Created aarecords__6.json
🔄 Decompressing aarecords__7.json.gz...
✅ Created aarecords__7.json
🔄 Decompressing aarecords__4.json.gz...
✅ Created aarecords__4.json
🔄 Decompressing aarecords__3.json.gz...
✅ Created aarecords__3.json
🔄 Decompressing aarecords__0.json.gz...
✅ Created aarecords__0.json
🔄 Decompressing aarecords__9.json.gz...
✅ Created aarecords__9.json
🔄 Decompressing aarecords__2.json.gz...
✅ Created aarecords__2.json
🔄 Decompressing aarecords__10.json.gz...
✅ Created aarecords__10.json
🔄 Decompressing aarecords__5.json.gz...
✅ Created aarecords__5.json


## 2. Convert JSON to Parquet in Chunks
If you have a lot of ram, you can reduce the number of batches per file, with 32 I was fine with 10, but you can try to reduce it.

In [3]:
def convert_to_parquet(json_file):
    print(f"\n🔄 Processing {os.path.basename(json_file)}")
    
    # Connect to DuckDB
    con = duckdb.connect()
    
    # Count total rows
    total_rows = con.execute(f"SELECT COUNT(*) FROM read_ndjson_objects('{json_file}');").fetchone()[0]
    chunk_size = total_rows // 10
    print(f"📊 Total rows: {total_rows} | Chunk size: {chunk_size}")
    
    # Process in 10 chunks
    offset = 0
    base_name = os.path.basename(json_file)
    file_name = os.path.splitext(base_name)[0]
    
    for i in range(10):
        print(f"🔄 Processing chunk {i+1}/10 (Offset: {offset})...")
        
        query = f"""
        CREATE OR REPLACE TABLE batch_data AS
        SELECT * FROM read_ndjson_objects('{json_file}')
        LIMIT {chunk_size} OFFSET {offset};
        """
        con.execute(query)
        
        count = con.execute("SELECT COUNT(*) FROM batch_data;").fetchone()[0]
        if count == 0:
            print("✅ No more rows to process.")
            break
        
        # Save chunk as Parquet
        parquet_file = f"/data/parquet/{file_name}_{i+1}.parquet"
        con.execute(f"COPY batch_data TO '{parquet_file}' (FORMAT 'parquet', COMPRESSION 'zstd');")
        print(f"✅ Saved: {os.path.basename(parquet_file)} ({count} rows)")
        
        offset += count
    
    con.close()
    print(f"✅ Completed processing {base_name}")

# Process all JSON files
json_files = glob.glob('/data/json/*.json')
for json_file in json_files:
    convert_to_parquet(json_file)


🔄 Processing aarecords__7.json


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

📊 Total rows: 3987464 | Chunk size: 398746
🔄 Processing chunk 1/10 (Offset: 0)...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Saved: aarecords__0_9.parquet (398973 rows)
🔄 Processing chunk 10/10 (Offset: 3590757)...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Saved: aarecords__0_10.parquet (398973 rows)
✅ Completed processing aarecords__0.json
